In [1]:
import torch
import random
import warnings
warnings.filterwarnings("ignore")

# 1. Define the Model Class
class LSTMModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, dropout=0.5, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x, mask):
        out, _ = self.lstm(x)
        last_indices = mask.sum(dim=1).long() - 1
        last_indices = last_indices.clamp(min=0)
        batch_size = x.size(0)
        out = out[torch.arange(batch_size), last_indices, :]
        out = self.fc(out)
        return out.squeeze()

# 2. Function to Generate a Fake Individual (Latest Version)
def generate_fake_individual(input_dim=25, total_years=10, min_years=1, max_years=10):
    """
    Generate a fake individual's career with random number of observed years.
    """
    available_years = list(range(total_years))
    
    # Random number of years for career (between 2 and 9)
    observed_years = random.randint(min_years, max_years)
    
    selected_years = sorted(random.sample(available_years, observed_years))
    data = []
    for year in selected_years:
        features = torch.randn(input_dim)  # Standard normal distribution (mean=0, std=1)
        data.append(features)
    
    return torch.stack(data)  # Shape: (observed_years, input_dim)

# 3. Function to Predict Single Individual
def predict_single_individual(model_path, network_sequence, input_dim, hidden_dim):
    """
    Load a model and predict the probability for a single individual's career sequence.
    """
    model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Prepare input
    network_sequence = network_sequence.unsqueeze(0)  # (1, seq_len, input_dim)
    mask = torch.ones((1, network_sequence.shape[1]))  # (1, seq_len)

    with torch.no_grad():
        output = model(network_sequence, mask)
        prob = torch.sigmoid(output)

    return prob.item()

# 4. Full Workflow

# Parameters
input_dim = 25  # 5 channels × 5 measures
hidden_dim = 64
model_path = "./all_all/model_valyear_2019.pth" 

# Step 1: Generate a fake individual's network career
sample_network_sequence = generate_fake_individual(
    input_dim=input_dim,
    total_years=10,
    min_years=2,
    max_years=9
)

# Step 2: Predict using the saved model
predicted_probability = predict_single_individual(
    model_path=model_path,
    network_sequence=sample_network_sequence,
    input_dim=input_dim,
    hidden_dim=hidden_dim
)

# Step 3: Print the result
print(f"Predicted probability of board appointment: {predicted_probability:.4f}")


Predicted probability of board appointment: 0.1402
